# Adattisztítás és integráció

A valós adathalmazok gyakran zajosak, hiányosak, avagy éppen redundáns információt vagy duplikátum egyedeket tartalmaznak. Ezért a tudásfeltárás folyamatában az adattisztítás és adatintegrálással kezdődik.

Az adattisztítás szerepe javítani az adatok minőségén azáltal, hogy kiszűri és eltávolítja az adatokban fellépő hibákat és inkonzisztenciákat.

Az [adattisztítás](https://hu.wikipedia.org/wiki/Adattiszt%C3%ADtás) során:
- felmérjük a hibákat
	- ellenőrizzük az adatfájl szerkezeti épségét
	- a zajt, felesleges információt tartalmazó mezőket javítjuk
	- felmérjük a hiányzó értékeket és amennyiben lehet ezeket pótoljuk
	- felmérjük az adatközlési és adatbeviteli hibákat
		- megvizsgáljuk az egyes változók eloszlását
			- az eloszlások szélein elhelyezkedő extrém értékeket ellenőrizzük
			- felmérjük, hogy az eloszlások megfelelnek-e az előzetes elvárásainknak, vannak-e nem várt sűrűsödések, ritkulások egyes értéktartományokban (például durva kerekítés vagy eltérő mértékegység használata az adatszolgáltatók egy részénél)
		-  megvizsgáljuk, hogy a változók közötti triviális összefüggések teljesülnek-e
-  a hibásnak tűnő adatokat felülvizsgáljuk, javítjuk.

# Feladatok

Az adatok letöltésére használjuk a `requests` könyvtárat. A requests könyvtár lehetővé teszi, hogy az internetről HTTPS protokollal bármilyen állományt letöltsünk, beleértve HTML dokumentumokat is. Sokszor megtörténhet, hogy az adatokat egy weboldalról kell letöltenünk, majd tisztítanunk valamely módon.

A HTML-es adatok feldolgozására használjuk a `BeautifulSoup4` könyvtárat. A `BeautifulSoup4` könyvtár segítségével fel tudjuk dolgozni a HTML adatokat. A könyvtár lehetővé teszi, hogy betöltsük a memóriába a HTML-ből vett struktúrákat, és általánosan tudjunk benne keresni elemeket, kiolvasni attribútumokat, stb.

1. A requests, BeautifulSoup4, plotly és nbformat könyvtárak nem képezik részét a Python standard könyvtárkészletének. Töltsük le a pip package managerrel ezeket a könyvtárakat.

In [ ]:
#!python -m pip install requests beautifulsoup4 plotly nbformat --upgrade --user

## Amerikai egyetemek

1. Az `egyetemek.txt` fájlból szűrjük ki az államokat és azon belül a városokat, melyben egyetemek találhatóak. Ha vannak duplikátumok, helytelen adatok (pl. számokat tartalmazó államnév), ezeket javítsuk. Vizsgáljuk meg az egyetemek eloszlását államok szerint.

In [ ]:
from collections import namedtuple
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

path='/content/drive/My Drive/Colab Notebooks/egyetemek.txt'

Item = namedtuple('Item', 'state city university')
items = []
i=0
nums = '0123456789'

def hasNum(text):
  for ch in text:
    if ch in nums:
      return True
    return False

with open(path,'r') as f:
    for line in f:
        clear_line = line.rstrip('\n')
        if clear_line.endswith('[edit]'):
          if hasNum(clear_line):
            continue
          state = clear_line.rstrip('[edit]')
        else:

          start_index = clear_line.find('(')
          end_index = clear_line.find(')')

          city = clear_line[:start_index-2]
          if hasNum(city):
            continue

          universities = clear_line[start_index:]
          universities = universities[1:end_index]
          universities = universities.split(")")
          universities = universities[0]
          cutted_universities = universities.split(", ")
          for uni in cutted_universities:
            if hasNum(uni):
              continue
            items.append(Item(state, city, uni))


          i = i+1

df = pd.DataFrame.from_records(items, columns=['State', 'City', 'Universities'])

#print(df[df.State == "Alabama"])
print(df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         State          City                           Universities
0      Alabama         Aubur                      Auburn University
1      Alabama       Florenc            University of North Alabama
2      Alabama   Jacksonvill          Jacksonville State University
3      Alabama     Livingsto             University of West Alabama
4      Alabama     Montevall               University of Montevallo
..         ...           ...                                    ...
728  Wisconsin    River Fall    University of Wisconsin–River Falls
729  Wisconsin  Stevens Poin  University of Wisconsin–Stevens Point
730  Wisconsin       Waukesh                     Carroll University
731  Wisconsin     Whitewate     University of Wisconsin–Whitewater
732    Wyoming        Larami                  University of Wyoming

[733 rows x 3 columns]


2. Melyik államban van a legtöbb, legkevesebb egyetem?

In [ ]:
def most_uni(df):
  return df.groupby(df.State)['Universities'].count().idxmax()

def leas_uni(df):
  return df.groupby(df.State)['Universities'].count().idxmin()

In [ ]:
most = most_uni(df)
print(most)

least = leas_uni(df)
print(least)

New York
Alaska


3. Bővítsük ki az adatbázisunkat egy oszloppal, mely tartalmazza az államok rövidítését is (pl. Texas - TX, California - CA stb.).  [Forrás](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

In [ ]:
# Importáljuk a könyvtárakat
import pandas as pd
import requests
import bs4

# Álcázzuk magunkat egy webböngészőnek
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
headers = {'User-Agent': user_agent}

url = 'https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations'

# Töltsük le a weboldalt
with requests.get(url, headers=headers) as req:
    # Ha nem sikerült a letöltés, akkor hibát dobunk
    req.raise_for_status()

    # A letöltött HTML-t lementjük egy változóba
    html_data = req.text

In [ ]:
# A BeautifulSoup segítségével dolgozzuk fel a letöltött HTML-t
soup = bs4.BeautifulSoup(html_data, features='html.parser')

# Keresük meg az oldalon azt a táblázatot, aminek az osztálya 'wikitable sortable'
table = soup.find('table', {'class': 'wikitable sortable'})

# A táblázatban keressük meg a sorokat
table_rows = table.find_all('tr')

# Ide fogjuk menteni az államok nevét és rövidítését
states = []
abbreviations = []

for table_row in table_rows:
    # A sorokon belül keressük meg az oszlopokat
    table_cols = table_row.find_all('td')

    # Ha az oszlopok száma 10, akkor egy államról van szó
    if len(table_cols) == 10:
        # Az állam neve a 0. oszlopban található
        state = table_cols[0].text

        # Az állam rövidítése a 3. oszlopban található
        abbreviation = table_cols[3].text

        if not abbreviation:
            # Ha nincs rövidítése az államnak, akkor a következő sorra ugrunk
            continue

        states.append(state)
        abbreviations.append(abbreviation)

# Hozzuk létre a DataFrame-et
state_abbreviations = pd.DataFrame({'State': states, 'Abbreviation': abbreviations})
state_abbreviations

,State,Abbreviation
0,United States of America,US
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA
6,Colorado,CO
7,Connecticut,CT
8,Delaware,DE
9,District of Columbia,DC


In [ ]:
# Rövidítések hozzáadása
import pandas as pd

df['State'] = df['State'].str.strip()
state_abbreviations['State'] = state_abbreviations['State'].str.strip()

df = pd.merge(df, state_abbreviations, on='State')


print(df[['Abbreviation', 'State', 'City', 'Universities']])

    Abbreviation      State          City  \
0             AL    Alabama         Aubur   
1             AL    Alabama       Florenc   
2             AL    Alabama   Jacksonvill   
3             AL    Alabama     Livingsto   
4             AL    Alabama     Montevall   
..           ...        ...           ...   
626           WI  Wisconsin    River Fall   
627           WI  Wisconsin  Stevens Poin   
628           WI  Wisconsin       Waukesh   
629           WI  Wisconsin     Whitewate   
630           WY    Wyoming        Larami   

                              Universities  
0                        Auburn University  
1              University of North Alabama  
2            Jacksonville State University  
3               University of West Alabama  
4                 University of Montevallo  
..                                     ...  
626    University of Wisconsin–River Falls  
627  University of Wisconsin–Stevens Point  
628                     Carroll University  
629     U

4. Hasonlóan, a [List of U.S. states and territories by area](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area) linken szereplő területi adatokat is integráljuk be a datasetünkbe.

In [ ]:
import pandas as pd
import requests
import bs4

# Álcázzuk magunkat egy webböngészőnek
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
headers = {'User-Agent': user_agent}

url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area'

# Töltsük le a weboldalt
with requests.get(url, headers=headers) as req:
    # Ha nem sikerült a letöltés, akkor hibát dobunk
    req.raise_for_status()

    # A letöltött HTML-t lementjük egy változóba
    html_data = req.text

In [ ]:
# A BeautifulSoup segítségével dolgozzuk fel a letöltött HTML-t
soup = bs4.BeautifulSoup(html_data, features='html.parser')
# Keresük meg az oldalon azt a táblázatot, aminek az osztálya 'wikitable sortable'
table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers col1left'})


# A táblázatban keressük meg a sorokat
table_rows = table.find_all('tr')

# Ide fogjuk menteni az államok nevét és rövidítését
states = []
total_area = []
land_area = []
water_area = []



for table_row in table_rows:
    # A sorokon belül keressük meg az oszlopokat
    table_cols = table_row.find_all('td')

    # Ha az oszlopok száma 9, akkor egy államról van szó
    if len(table_cols) == 8:
        # Az állam neve a 1. oszlopban található
        state = table_cols[0].text
        if 'Contiguous US' in state:
          break
        # a total area km^2 3. oszlop
        total = table_cols[2].text
        # a land area km^2 5. oszlop
        land = table_cols[4].text
        # a water area km^2 7. oszlop
        water = table_cols[6].text

        states.append(state)
        total_area.append(pd.to_numeric(total.replace(',', '')))
        land_area.append(pd.to_numeric(land.replace(',', '')))
        water_area.append(pd.to_numeric(water.replace(',', '')))

# Hozzuk létre a DataFrame-et
state_area = pd.DataFrame({'State': states, 'Total': total_area, 'Land': land_area, 'Water': water_area})
state_area

,State,Total,Land,Water
0,Alaska,1723337,1477953,245383
1,Texas,695662,676587,19075
2,California,423967,403466,20501
3,Montana,380831,376962,3869
4,New Mexico,314917,314161,757
5,Arizona,295234,294207,1026
6,Nevada,286380,284332,2048
7,Colorado,269601,268431,1170
8,Oregon,254799,248608,6191
9,Wyoming,253335,251470,1864


In [ ]:
# Területek hozzáadása
import pandas as pd

state_area['State'] = state_area['State'].str.strip()

df = pd.merge(df, state_area, on='State')


print(df)

In [ ]:
df.head()

,State,City,Universities,Abbreviation,Total,Land,Water
0,Alabama,Aubur,Auburn University,AL,135767,131171,4597
1,Alabama,Florenc,University of North Alabama,AL,135767,131171,4597
2,Alabama,Jacksonvill,Jacksonville State University,AL,135767,131171,4597
3,Alabama,Livingsto,University of West Alabama,AL,135767,131171,4597
4,Alabama,Montevall,University of Montevallo,AL,135767,131171,4597


5. Számoljuk ki államonként átlagban hány négyzetkilométerre jut egy egyetemi város.

In [ ]:
import pandas as pd

def avg_uni_area(df):
  grouped = df.groupby('State')
  size = grouped.size()

  average_area_per_city = grouped['Total'].mean() / size
  return average_area_per_city.astype(int)

print(avg_uni_area(df))

State
Alabama             13576
Alaska            1723337
Arizona             98411
Arkansas            12521
California           7570
Colorado            29955
Florida              9461
Georgia              7329
Idaho               72147
Illinois            14999
Indiana              6737
Iowa                11211
Kansas              26637
Kentucky             8050
Louisiana           15073
Massachusetts         650
Michigan            10890
Minnesota            9381
Montana            126943
Nebraska            28618
Nevada              71595
New Jersey           2259
New Mexico          62983
New York             2207
North Carolina       5361
North Dakota        91554
Ohio                 5047
Oklahoma            15086
Oregon              19599
Pennsylvania         2168
South Carolina       3949
South Dakota        49932
Texas               21739
Utah                36647
Virginia             4431
Washington          36932
West Virginia        6972
Wisconsin           10602
Wyomin

6. Az ábrán a kivitel (exportmennyiség) van ábrázolva az Amerikai Egyesült Államok térképen, államonként lebontva. Készítsünk hasonló ábrákat az egyetemek eloszlásáról is. A [térképen való ábrázolás](https://plotly.com/python/maps/) Plotly segítségével történhet.

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

fig = go.Figure(data=go.Choropleth(
	locations=df['code'], # államok kódja,
	z = df['total exports'].astype(float), # adat, amit színezni akarunk
	locationmode = 'USA-states',
	colorscale = 'Reds',
	colorbar_title = 'Millions USD'
))
fig.update_layout(
	title_text = '2011 US Agriculture Exports by State',
	geo_scope='usa' # csak az USA-t jelenítsük meg
)
fig.show()

## Vásárhelyi éttermek

Keressük meg Vásárhelyen a legolcsóbb és legdrágább pizzákat. A pizzákat keressük meg az [eeatingh.ro](https://eeatingh.ro) oldalán. Az [eeatingh.ro](https://eeatingh.ro) oldalon több étterem található, mindegyik étterem közzé teszi a saját menüjét.

![image.png](attachment:image.png)

1. Használjuk a requests könyvtárat, hogy letöltsük az éttermek listáját a főoldalról. Mentsük le az éttermek nevét (például: Rexo LunchBox), illetve az étteremhez tartozó adatlap elérhetőségét (például: `https://eeatingh.ro/targu-mures/rexo-lunchbox.html`) egy DataFramebe. Mentsük ki a DataFrame-et egy állományba, hogy később gyorsabban lehessen az adatokat feldolgozni!

Segítség: Használjuk a Chrome-ban a beépített `Inspect Element` funkcionalitást. Keressük meg a HTML dokumentumban, hogy hol vannak lementve az éttermek. Minden éttermet külön entitásnak tekinthetünk. Kerssük meg, hogy egy éttermen belül hol van lementve az étterem neve, illetve az étteremhez tartozó adatlap elérhetősége. Ezeket az információkat használjuk fel arra, hogy megírjuk a HTML feldolgozását a BeautifulSoup4 segítségével.

In [2]:
#eeathing éttermek

# Importáljuk a könyvtárakat
from collections import namedtuple
import pandas as pd
import requests
import bs4
import csv
import re
from google.colab import drive

drive.mount('/content/drive')

# Álcázzuk magunkat egy webböngészőnek
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
headers = {'User-Agent': user_agent}

url_food = 'https://eeatingh.ro/targu-mures.html'

with requests.get(url_food, headers=headers) as req:
    # Ha nem sikerült a letöltés, akkor hibát dobunk
    req.raise_for_status()

    # A letöltött HTML-t lementjük egy változóba
    html_food_data = req.text

# A BeautifulSoup segítségével dolgozzuk fel a letöltött HTML-t
soup_food = bs4.BeautifulSoup(html_food_data, features='html.parser')

all_restaurants = soup_food.find('div', {'class': 'listStoresBox'})

rest_names = []
rest_links = []

if all_restaurants:
    # print("van")
    # Az összes étterem nevének kinyerése
    allRestaurants_divs = all_restaurants.find_all('a', class_= 'storeItem')
    # print(allRestaurants_divs)
    for div in allRestaurants_divs:
        span = div.find('span', {'class': 'name semibold'})
        link = div.get('href')
        if span:
            rest_names.append(span.text.strip())
            rest_links.append(link)
            # print(span.text.strip())
            # print(link)

# print(rest_names)
# print(rest_links)

foodItem = namedtuple('Item', 'restaurant_name url food_name description price')
foodItems = []
i = 0

for link,name in zip(rest_links, rest_names):
    print(link)
    print(name)
    try:
      with requests.get(link, headers=headers) as req:
        # Ha nem sikerült a letöltés, akkor hibát dobunk
        req.raise_for_status()

        # A letöltött HTML-t lementjük egy változóba
        html_one_food_data = req.text

      soup_one_food = bs4.BeautifulSoup(html_one_food_data, features='html.parser')

      # content = soup_one_food.find('div', {'class': 'content'})
      # store_box = content.find('div', {'class': 'storeBox'})
      # storeWrapper_template1 = store_box.find('div',{'class': 'storeWrapper template1'})
      content = soup_one_food.find('div', {'class': 'storeContent'})
      storeContent = content.find_all('div', {'class': 'productWrap'})
      # print(storeContent)
      for food in storeContent:
        foodNameDiv = food.find('div', {'class': 'productInfoWrap'})
        foodName = foodNameDiv.find('h1', {'class': 'productTitle semibold'})
        foodName = foodName.text
        foodDescription = foodNameDiv.find('div', {'class': 'productDescription'})

        foodMoreDescription = foodNameDiv.find('a', {'class': 'moreDetails'})
        if foodMoreDescription:
          foodDescription = foodMoreDescription.get('title')
        else:
          foodDescription = foodDescription.text

        productSizesWrap = foodNameDiv.find('div', {'class': 'productSizesWrap'})
        prices = productSizesWrap.find_all('a')
        for price in prices:
          small_desc = price.find('span', {'class': 'descriptionInfo'})
          if small_desc:
            newFoodName = foodName + small_desc.text
          else:
            newFoodName = foodName
          foodPrice = price.find('span', {'class': 'amount bold'})
          foodPrice = foodPrice.text
          foodPrice = re.findall(r'\d+', foodPrice)
          foodPrice = ''.join(foodPrice)
          price_as_int = int(foodPrice)
          foodItems.append(foodItem(name,link,newFoodName, foodDescription, foodPrice))
        print('-------------------------------------------------------------------------------------------')
        print(newFoodName, '    ', foodPrice)
        print(foodDescription)
    except requests.exceptions.HTTPError as err:
      if req.status_code == 404:
        continue

print(foodItems)


new_path = '/content/drive/My Drive/Colab Notebooks/foodInfo.csv'
with open(new_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['restaurant_name', 'url', 'food_name', 'description', 'price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for food_info in foodItems:
        writer.writerow({
            'restaurant_name': food_info.restaurant_name,
            'url': food_info.url,
            'food_name': food_info.food_name,
            'description': food_info.description,
            'price': food_info.price
        })





Streaming output truncated to the last 5000 lines.
-------------------------------------------------------------------------------------------
White Mocca Classico 250ml      10
Espresso 30 ml, cremă de lapte 200 ml, sirop de ciocolată albă 20 ml
-------------------------------------------------------------------------------------------
White Mocca Medium 350ml      12
espresso 30 ml, crema de lapte 300 ml, sirop ciocolata alba 20 ml
-------------------------------------------------------------------------------------------
Vanilla Cinnamon Latte 250 ml      10
Espresso 30 ml, cremă de lapte 200 ml, sirop de vanilie si  scorțișoara 20 ml
-------------------------------------------------------------------------------------------
Toffe Crunch choco 250 ml      11
Ciocolată caldă clasică 30 g , lapte 200 , sirop de toffee crunch 20 ml
-------------------------------------------------------------------------------------------
White Mocca Grande 400ml      14
Espresso 30 ml, cremă de lapte 

2. Éttermenként töltsük le a vendeglők adatlapját. Az étterem adatlapján megtalálható az adott vendéglő menüje. Haladjunk végig az összes éttermen, töltsük le őket, és keressük ki a rendelhető fogásokat. Mentsük le az ételek / italok nevét, árát és leírását, illetve az étterem nevét, ahol rendelni lehet őket. Ezeket az információkat mentsük ki egy DataFrame-be. Mentsük ki a DataFrame-et egy állományba, hogy később gyorsabban lehessen az adatokat feldolgozni!

Segítség: Mindegyik rendelhető étel felfogható egy külön entitásként. A rendelhető ételek egy-egy div-nek felelnek meg a HTML forrásállományban. Használjuk a Chrome `Inspect Element` funkcionalitását, hogy találjuk meg azt az attribútumot, amivel ki lehet szűrni a rendelhető fogásokat. Az étel nevét és árát ugyancsak ki lehet szedni egy-egy HTML elemből.

![image.png](attachment:image.png)

In [3]:
new_path = '/content/drive/My Drive/Colab Notebooks/foodInfo.csv'
df2 = pd.read_csv(new_path)
df2

,restaurant_name,url,food_name,description,price
0,Rexo LunchBox,https://eeatingh.ro/targu-mures/rexo-lunchbox....,Cheesy Chicken Fries 330 g,"cartofi pai 170 g, piept de pui 80 g, mozzarel...",22
1,Rexo LunchBox,https://eeatingh.ro/targu-mures/rexo-lunchbox....,Quattro Stagioni Standard,"blat pizza, sos pizza, mozzarella, șuncă, ciup...",30
2,Rexo LunchBox,https://eeatingh.ro/targu-mures/rexo-lunchbox....,Crispy Shaorma 400 g,"crispy din piept de pui 120 g, lipie 60 g, sal...",236
3,Rexo LunchBox,https://eeatingh.ro/targu-mures/rexo-lunchbox....,Kung Pao Rice Box 400 g,"carne la alegere, ceapă, ghimbir, usturoi, ard...",284
4,Rexo LunchBox,https://eeatingh.ro/targu-mures/rexo-lunchbox....,Curry Ginger Noodle Box 400 g,"piept de pui, ciuperci, morcovi, ceapă, ardei,...",316
...,...,...,...,...,...
8239,Sprit Delivery,https://eeatingh.ro/targu-mures/sprit-delivery...,Aurelia Visinescu Nomad Merlot,Vin rosu,34
8240,Sprit Delivery,https://eeatingh.ro/targu-mures/sprit-delivery...,Liliac Feteasca Neagra,"Vin rosu, sec",65
8241,Sprit Delivery,https://eeatingh.ro/targu-mures/sprit-delivery...,Corona,"Bere Corona, 4%, 0.335",7
8242,Sprit Delivery,https://eeatingh.ro/targu-mures/sprit-delivery...,Paulaner Weissbier,"Bere Paulaner Weiss, 0.5l",12


3. Keressük meg az összes olyan ételt, aminek a leírásában megtalálható a `pizza` szócska. Ezeket az ételeket rendezzük növekvő sorrendbe az áruk alapján. Másnéven: keressük meg Vásárhelyen a legolcsóbb és legdrágább pizzákat.

4. Keressük meg az összes olyan ételt, ami 30 lej alatt megrendelhető.

5. Melyik a legdrágább étel az oldalon?

6. Melyik étteremnek van a legnagyobb kínálata?

7. Számoljuk ki vendéglőnként az ételek átlag árát! Melyik étterem a legdrágább / legolcsóbb?